In [0]:
USE CATALOG hytech_workshop;
USE SCHEMA ai_agent;

In [0]:
CREATE OR REPLACE TABLE customers
USING DELTA
AS
SELECT * 
FROM read_files(
  '/Volumes/hytech_workshop/ai_agent/raw_data/customers',
  format => 'parquet',
  schemaEvolutionMode => 'none'
);

CREATE OR REPLACE TABLE billing
USING DELTA
AS
SELECT * 
FROM read_files(
  '/Volumes/hytech_workshop/ai_agent/raw_data/billing',
  format => 'parquet',
  schemaEvolutionMode => 'none'
);

CREATE OR REPLACE TABLE subscriptions
USING DELTA
AS
SELECT * 
FROM read_files(
  '/Volumes/hytech_workshop/ai_agent/raw_data/subscriptions',
  format => 'parquet',
  schemaEvolutionMode => 'none'
);

In [0]:
CREATE OR REPLACE FUNCTION get_customer_by_email(email_input STRING COMMENT 'customer email used to retrieve customer information')
RETURNS TABLE (
    customer_id BIGINT,
    first_name STRING,
    last_name STRING,
    email STRING,
    phone STRING,
    address STRING,
    city STRING,
    state STRING,
    zip_code STRING,
    customer_segment STRING,
    registration_date DATE,
    customer_status STRING,
    loyalty_tier STRING,
    tenure_years DOUBLE,
    churn_risk_score BIGINT,
    customer_value_score BIGINT
)
COMMENT 'Returns the customer record matching the provided email address. Includes its ID, firstname, lastname and more.'
RETURN (
    SELECT * FROM customers
    WHERE email = email_input
    LIMIT 1
);

In [0]:
SELECT * FROM get_customer_by_email('john21@example.net');

In [0]:
CREATE OR REPLACE FUNCTION get_customer_billing_and_subscriptions(customer_id_input BIGINT COMMENT 'customer ID used to retrive orders, billing and subscriptiosn')
RETURNS TABLE (
    customer_id BIGINT,
    subscription_id BIGINT,
    service_type STRING,
    plan_name STRING,
    plan_tier STRING,
    monthly_charge BIGINT,
    start_date DATE,
    contract_length_months BIGINT,
    status STRING,
    autopay_enabled BOOLEAN,
    total_billed DOUBLE,
    total_paid DOUBLE,
    total_late_payments BIGINT,
    total_late_fees DOUBLE,
    latest_payment_status STRING
)
COMMENT 'Returns subscription and billing details for a customer.'
RETURN (
    SELECT
        s.customer_id, s.subscription_id, s.service_type, s.plan_name, s.plan_tier,
        s.monthly_charge, s.start_date, s.contract_length_months, s.status, s.autopay_enabled,
        COALESCE(b.total_billed, 0), COALESCE(b.total_paid, 0),
        COALESCE(b.total_late_payments, 0), COALESCE(b.total_late_fees, 0),
        COALESCE(b.latest_payment_status, 'N/A')
    FROM subscriptions s
    LEFT JOIN (
        SELECT
            subscription_id, customer_id,
            SUM(total_amount) AS total_billed,
            SUM(payment_amount) AS total_paid,
            COUNT_IF(payment_date > due_date OR payment_status = 'Late') AS total_late_payments,
            SUM(CASE WHEN payment_date > due_date OR payment_status = 'Late' THEN total_amount - payment_amount ELSE 0 END) AS total_late_fees,
            MAX(payment_status) AS latest_payment_status
        FROM billing
        WHERE customer_id = customer_id_input
        GROUP BY subscription_id, customer_id
    ) b ON s.subscription_id = b.subscription_id
    WHERE s.customer_id = customer_id_input
);

In [0]:
SELECT *
FROM get_customer_billing_and_subscriptions(
  (SELECT customer_id FROM get_customer_by_email('john21@example.net'))
);